In [1]:
%%time

import pandas as pd
from pymongo import MongoClient
import json

dbuser = 'rest_user'
dbpass = 'itversity'
dbhost = 'restmongo'
dbport = 27017

client = MongoClient(f'mongodb://{dbuser}:{dbpass}@{dbhost}:{dbport}')

db = client['restdb']

coll = db['restaurant_ratings']

rest_chunks = pd.read_csv('/data/train_full.csv', chunksize=10000)

for rest_df in rest_chunks:
    print('.', end='')
    rest_recs = json.loads(
        rest_df. \
            reset_index(). \
            rename(columns={'index':'rating_id'}). \
            to_json(orient='records')
    )
    coll.insert_many(rest_recs)

...................................................................................................................................................................................................................................................................................

/home/itversity/.local/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


...........................................................................................................................................................................................................................................................................................

/home/itversity/.local/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


.......................CPU times: user 7min 30s, sys: 18.4 s, total: 7min 49s
Wall time: 8min 57s


In [3]:
%%time
coll.create_index('rating_id')

CPU times: user 13.2 ms, sys: 5.24 ms, total: 18.4 ms
Wall time: 22.3 s


'rating_id_1'

* Validating read performance

In [11]:
%%time
import random
for i in range(10000):
    rating_id = random.randint(1, 5000000)
    rec = coll.find_one({'rating_id': rating_id})

CPU times: user 3.17 s, sys: 605 ms, total: 3.77 s
Wall time: 8.06 s


* Validating update performance

In [12]:
%%time
import random
for i in range(10000):
    rating_id = random.randint(1, 5000000)
    rec = coll.find_one({'rating_id': rating_id})
    coll.update_one({'rating_id': rating_id}, {'$set': {'customer_id': rec['customer_id']}})

CPU times: user 6.62 s, sys: 1.38 s, total: 8 s
Wall time: 15.8 s


* Validating delete performance

In [13]:
%%time
import random
for i in range(10000):
    rating_id = random.randint(1, 5000000)
    coll.delete_one({'rating_id': rating_id})

CPU times: user 2.17 s, sys: 494 ms, total: 2.67 s
Wall time: 8.58 s
